In [1]:
import pandas as pd
import datetime
import stock_data as stock
import numpy as np
from tqdm import tqdm

### Get Data

import stock_data as stock

scrapy = stock.Scrapy()
today = datetime.datetime.today()
end = f"{today.year}-12-31"

price = scrapy.get_price(
    start = "2013-01-01",
    end = end,
    mode = "listed"
)
price.to_csv("data/price_2013_2023.csv", index = False)
print(f"length = {len(price)}")
price.head()

twiis = scrapy.get_price(
    start = "2013-01-01",
    end = end,
    mode = "other",
    query = "^TWII"
)
twiis.to_csv("data/twii_2013_2023.csv", index = False)

ratio = scrapy.get_financial_statement(
    type_ = 3,
    clean = 1, 
    mode = "listed",
    start_year = today.year-1911-10, 
    end_year = today.year-1911-1
)
ratio.to_csv("data/ratio_2013_2022.csv", index = False)
print(f"length = {len(ratio)}")
ratio.head()

income = scrapy.get_financial_statement(
    type_ = 1,
    clean = 1, 
    mode = "listed",
    start_year = today.year-1911-10, 
    end_year = today.year-1911-1
)
income.to_csv("data/income_2013_2022.csv", index = False)
print(f"length = {len(income)}")
income.head()

asset = scrapy.get_financial_statement(
    type_ = 2,
    clean = 1, 
    mode = "listed",
    start_year = today.year-1911-10, 
    end_year = today.year-1911-1
)
asset.to_csv("data/asset_2013_2022.csv", index = False)
print(f"length = {len(asset)}")
asset.head()

### Load Data

In [2]:
twii_raw = pd.read_csv("data/twii_2013_2023.csv", parse_dates = ["Date"])
price_raw = pd.read_csv("data/price_2013_2023.csv", parse_dates = ["Date"])
ratio_raw = pd.read_csv("data/ratio_2013_2022.csv")
income_raw = pd.read_csv("data/income_2013_2022.csv")
asset_raw = pd.read_csv("data/asset_2013_2022.csv")
print(f"twii_raw length = {len(twii_raw)}")
print(f"price length = {len(price_raw)}")
print(f"ratio length = {len(ratio_raw)}")
print(f"income_raw length = {len(income_raw)}")
print(f"asset_raw length = {len(asset_raw)}")

twii_raw length = 2617
price length = 2388161
ratio length = 35151
income_raw length = 36353
asset_raw length = 36353


#### Clean

In [3]:
def price(df):
    df = df.drop(["Open", "High", "Low", "Close"], axis = 1)
    df = df.rename(columns = {"Adj Close": "Close"})
    df["Year"] = df["Date"].dt.year
    return df

In [4]:
twii_raw = price(twii_raw)
twii_raw.head(3)

,Symbol,Date,Close,Volume,Year
0,^TWII,2013-01-02,7779.19,2194600,2013
1,^TWII,2013-01-03,7836.81,2964400,2013
2,^TWII,2013-01-04,7805.96,2696800,2013


In [5]:
price_raw["Symbol"] = price_raw["Symbol"].str.split(".").str[0]
price_raw = price_raw.sort_values(["Date", "Symbol"]).reset_index(drop = True)
price_raw = price(price_raw)
price_raw.head(3)

,Symbol,Date,Close,Volume,Year
0,1101,2013-01-02,16.79,8646455.0,2013
1,1102,2013-01-02,20.13,2955776.0,2013
2,1103,2013-01-02,9.09,319260.0,2013


In [6]:
def statement(df, keep_col, new_col_name):
    df = df.iloc[:, keep_col]
    df.columns = new_col_name
    df = df.sort_values(["Year", "Season", "Symbol"]).reset_index(drop = True)
    df["Symbol"] = df["Symbol"].astype(str)
    df["Year"] = df["Year"] + 1911

    return df

In [7]:
ratio_raw = statement(ratio_raw, keep_col = [0,1,2,3,4,5,6,8], new_col_name = ['Year', 'Season', 'Symbol', 'Name', "Sales", "Gross", "Operating", "Net"])
ratio_raw.head(3)

,Year,Season,Symbol,Name,Sales,Gross,Operating,Net
0,2013,1,1101,台泥,24114.05,12.95,8.40,8.29
1,2013,1,1102,亞泥,13931.55,6.40,2.44,9.84
2,2013,1,1103,嘉泥,741.19,-6.06,-20.21,8.05


In [8]:
income_raw = statement(income_raw, keep_col = [0,1,2,3,4,5,6,8,9,10], new_col_name = ['Year', 'Season', 'Symbol', 'Name', "Sales", "Gross", "Operating", "Net", "Total_Net", "EPS"])
income_raw.head(3)

,Year,Season,Symbol,Name,Sales,Gross,Operating,Net,Total_Net,EPS
0,2013,1,1101,台泥,24114047.0,3123872.0,2026729.0,1999624.0,3284069.0,0.38
1,2013,1,1102,亞泥,13931550.0,892180.0,339801.0,1371559.0,1246521.0,0.40
2,2013,1,1103,嘉泥,741189.0,-44905.0,-149811.0,59637.0,12307.0,0.13


In [9]:
asset_raw = statement(asset_raw, keep_col = [0,1,2,3,14], new_col_name = ['Year', 'Season', 'Symbol', 'Name', "Equity"])
asset_raw.head(3)

,Year,Season,Symbol,Name,Equity
0,2013,1,1101,台泥,145365698.0
1,2013,1,1102,亞泥,104251999.0
2,2013,1,1103,嘉泥,16395990.0


### Imputer time

In [10]:
def imputer_time(df, name):
    df1 = pd.DataFrame()
    g = df.groupby("Symbol")
    for group, df_group in tqdm(g, desc = name):
        combinations = df[["Year", "Season"]].value_counts().index
        combinations = combinations.sort_values().to_frame(index = False)
        df_group = pd.merge(combinations, df_group, on = ["Year", "Season"], how = "left")
        df1 = pd.concat([df1, df_group], ignore_index = True)
    df1[["Symbol", "Name"]] = df1[["Symbol", "Name"]].fillna(method = "ffill")    
    df1 = df1.sort_values(["Year", "Season", "Symbol"]).reset_index(drop = True)

    return df1

In [11]:
ratio_raw  = imputer_time(df = ratio_raw,  name = "ratio")
income_raw = imputer_time(df = income_raw, name = "income")
asset_raw  = imputer_time(df = asset_raw,  name = "asset")

print(f"ratio length = {len(ratio_raw)}")
print(f"income_raw length = {len(income_raw)}")
print(f"asset_raw length = {len(asset_raw)}")

asset: 100%|██████████| 983/983 [00:04<00:00, 226.58it/s]

ratio length = 38280
income_raw length = 39320
asset_raw length = 39320


### Remove cumulative

In [12]:
def remove_cumulative(df, name, col):
    g1 = df.groupby("Symbol")
    df_symbol = pd.DataFrame()
    for group1, df_group1 in tqdm(g1, desc = name):
        g2 = df_group1.groupby("Year")
        df_year = pd.DataFrame()
        
        for group2, df_group2 in g2:
            df_group2 = df_group2.reset_index(drop = True)
            df_group2_1 = df_group2[col] - df_group2[col].shift(1)
            df_group2_1.loc[0] = df_group2.loc[0, col]
            df_group2[col] = df_group2_1
            df_year = pd.concat([df_year, df_group2], ignore_index = True)

        df_symbol = pd.concat([df_symbol, df_year], ignore_index = True)

    df_symbol = df_symbol.sort_values(["Year", "Season", "Symbol"]).reset_index(drop = True)
    
    return df_symbol

In [13]:
ratio_raw  = remove_cumulative(df = ratio_raw,  name = "ratio",  col = ['Sales'])
income_raw = remove_cumulative(df = income_raw, name = "income", col = ['Sales', 'Gross', 'Operating', 'Net', 'Total_Net', 'EPS'])

income: 100%|██████████| 983/983 [00:22<00:00, 43.42it/s]


### Split year

In [14]:
def split_year(df, name):
    dfs = {}
    g = df.groupby("Year")
    for group, df_group in tqdm(g, desc = name):
        df_group = df_group.reset_index(drop = True)
        dfs[group] = df_group
    
    return dfs

In [15]:
prices  = split_year(df = price_raw,  name = "price")
twiis   = split_year(df = twii_raw,   name = "twii")
ratios  = split_year(df = ratio_raw,  name = "ratio")
incomes = split_year(df = income_raw, name = "income")
assets  = split_year(df = asset_raw,  name = "asset")

asset: 100%|██████████| 10/10 [00:00<00:00, 2498.69it/s]


### Filter stock by ratio

In [16]:
filter_stock = []
for year in tqdm(range(2015, 2023), desc = "year"):
    # 合併近三年財務比率
    ratio = pd.concat([ratios[year-2], ratios[year-1], ratios[year]])

    # Q4要隔年3月才會公布，只抓到Q3
    ratio = ratio.set_index(["Year", "Season"])
    ratio = ratio.loc[(year-2, 4) : (year, 3)].reset_index()
    # print(f"ratio length = {len(ratio)}")

    # 營收YOY為正，且三率三升
    ratio1 = pd.DataFrame()
    g = ratio.groupby("Symbol")
    for group, df_group in g:
        df_group = df_group.reset_index(drop = True)
        
        # 近8季 營收、毛利率、營業利益率、稅後淨利率 > 0
        remove_flag = (df_group.loc[:, "Sales":"Net"] < 0).any().any()
        if remove_flag:
            continue
        
        # 近四季 營收YOY > 0
        df_group["Sales_pct"] = df_group["Sales"].pct_change(4).values
        remove_flag = (df_group["Sales_pct"] < 0).any()
        if remove_flag:
            continue
        
        # 近四季 三率三升
        keep_flag = (df_group.loc[:3, "Sales":"Net"] < df_group.shift(-4).loc[:3, "Sales":"Net"]).all().all()
        if (keep_flag) and (len(df_group) >= 4):
            df_group = df_group.iloc[-4:].reset_index(drop = True)
            ratio1 = pd.concat([ratio1, df_group], ignore_index = True)
    ratio = ratio1.copy()
    # print(f"ratio length = {len(ratio)}")

    portfolio = ratio["Symbol"].unique()
    # print(f"Year: {year} (Count: {len(portfolio)})")
    # print(f"Portfolio: {portfolio}")

    filter_stock.append([year+1, len(portfolio), portfolio])


filter_stock = pd.DataFrame(filter_stock, columns = ["Year", "Port_Count", "Portfolio"])
filter_stock

year: 100%|██████████| 8/8 [00:07<00:00,  1.09it/s]


,Year,Port_Count,Portfolio
0,2016,18,"[1227, 1477, 1536, 1702, 1708, 2231, 2317, 238..."
1,2017,18,"[1319, 1568, 1752, 1789, 2228, 2345, 2355, 248..."
2,2018,16,"[1215, 1301, 1463, 1909, 2421, 2428, 2912, 300..."
3,2019,22,"[1102, 1109, 1256, 1301, 1457, 1597, 2049, 232..."
4,2020,17,"[1102, 1470, 1477, 1514, 2458, 2904, 3034, 341..."
5,2021,13,"[1210, 1732, 1786, 2313, 2383, 2449, 2454, 304..."
6,2022,57,"[1303, 1304, 1305, 1513, 1720, 1907, 1909, 200..."
7,2023,37,"[1475, 1477, 1524, 1560, 1708, 1712, 2049, 205..."


### Filter stock by income and asset

In [ ]:
for i in range(len(filter_stock)):
    year = filter_stock.loc[i, "Year"] - 1
    income = pd.concat([incomes[year-2], incomes[year-1], incomes[year]]) # 合併近三年財報
    portfolio = filter_stock.loc[i, "Portfolio"]
    income = income.query("Symbol in @portfolio").reset_index(drop = True)

    # Q4要隔年3月才會公布，只抓到Q3
    income = income.set_index(["Year", "Season"])
    income = income.loc[(year-2, 4) : (year, 3)].reset_index()
    # break

In [ ]:
# YOY接為正
income1 = pd.DataFrame()
g = income.groupby("Symbol")
for group, df_group in g:
    df_group = df_group.reset_index(drop = True)
    break

In [ ]:
df_group

### Caculate return

In [ ]:
portfolio_price = pd.DataFrame(columns = ["Year", "Symbol", "Start", "End", "Return"])
for i in range(len(filter_stock)):
    year = filter_stock.loc[i, "Year"]
    price = prices[year]
    portfolio = filter_stock.loc[i, "Portfolio"]
    # print(f"price length = {len(price)}")

    # 抓出篩選出的股票
    price = price.query("(Symbol in @portfolio) and (Close <= 100)")
    price = price.sort_values(["Symbol", "Date"]).reset_index(drop = True)
    # print(f"price length = {len(price)}")

    # 計算每檔股票的報酬率
    total_return = []
    price1 = pd.DataFrame()
    g = price.groupby("Symbol")
    groups = price["Symbol"].unique()
    for group in groups:
        df_group = g.get_group(group)
        start = df_group["Close"].iloc[0] # 年初
        end = df_group["Close"].iloc[-1] # 年底
        return_ = round((end - start) / start * 100, 2) # 報酬率
        total_return.append(return_)
        portfolio_price.loc[len(portfolio_price)] = [year, group, start, end, return_]

    filter_stock.loc[i, "Port_Return"] = round(np.mean(total_return), 2)

    # 加權指數
    twii = twiis[year]
    twii = twiis[year].sort_values(["Symbol", "Date"]).reset_index(drop = True)
    start = twii["Close"].iloc[0] # 年初
    end = twii["Close"].iloc[-1] # 年底
    return_twii = round((end - start) / start * 100, 2) # 報酬率
    filter_stock.loc[i, "TWII_Return"] = return_twii

filter_stock = filter_stock[['Year', 'Port_Return', 'TWII_Return', 'Port_Count', 'Portfolio']]

In [ ]:
filter_stock

In [ ]:
writer = pd.ExcelWriter('data/output.xlsx', engine = 'xlsxwriter')
filter_stock.to_excel(writer, sheet_name = 'performance', index = False)
portfolio_price.to_excel(writer, sheet_name = 'portfolio_price', index = False)
writer.close()